<a href="https://colab.research.google.com/github/ohmkas/Carteira-Quant-FI/blob/main/Gerar_Relat%C3%B3rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def descobre_cnpjs():
  execfile("/content/drive/MyDrive/Colab Notebooks/Carteira Quant/Gera Carteiras Dia/gera_carteiras_do_dia_dos_solicitantes.py")
  return (None)

descobre_cnpjs()

import plotly.graph_objects as go
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import RGBColor
from docx.shared import Pt
acima_cdi = []
def salvar(nome_pasta=str): #Cria ess pasta caso ela não exista ou simplemente já deixa o caminho da pasta pronto
  if not os.path.exists("/content/sample_data/"+nome_pasta): 
    os.mkdir("/content/sample_data/"+nome_pasta) 
  caminho = str("/content/sample_data/"+nome_pasta) 
  print('os arquivos serão salvos em:',caminho)
  return(caminho)

salvar_em = salvar(nome_pasta='Etapa Final/Relatorio')



try:
  with open('/content/sample_data/Etapa Final/CNPJS_SOLICITANTES.json', 'rb') as fp:
    cnpjs_solicitantes = pickle.load(fp)
except:
  pass

series = {}
for cliente in cnpjs_solicitantes:
  serie = cnpjs_solicitantes[cliente]
  serie_acum_max = ((1+(serie.pct_change())).cumprod()-1)
  serie_norm_dia = serie.dropna().pct_change()
  serie_norm_acum = (serie_norm_dia+1).cumprod()-1
  series.update({cliente:{'Acum. Total':serie_acum_max,'Normal Dia':serie_norm_dia,'Normal Acum': serie_norm_acum}})


  df_base=series[cliente]['Normal Acum']
  titulo='Evolução dos Fundos de Investimento sob critérios'
  subtitulo='Solicitante: {}'.format(cliente)
  unidade_y='Rentabilidade (%)'
  unidade_x='Data'

  fig = go.Figure()
  n=0
  annotations = []
  
  for item in df_base.columns:
    
    
    teste =  series[cliente]['Acum. Total'].dropna()[item].iloc[-1] > ((1+(cdi_graf['valor'].loc[df_base[item].dropna().index[0]:]/100)).cumprod()-1).iloc[-1]
    if teste == True:
      acima_cdi.append(item)
    else:
      pass
    fig.add_trace(go.Scatter(x=df_base[item].index,
                            y=df_base[item]*100,
                            name=item,
                            mode='lines',
                            showlegend=True,
                            line=dict(width=2),
                            line_shape='spline'))
    n+=1

  fig.add_trace(go.Scatter(x=cdi_graf.loc[df_base[item].index[0]:].index,
                            y=((1+(cdi_graf['valor'].loc[df_base[item].index[0]:]/100)).cumprod()-1)*100,
                            name='CDI',
                            mode='lines',
                            showlegend=True,
                            line=dict(width=2,dash='dot',color='black'),
                            line_shape='spline'))

  annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.10,
                                xanchor='left', yanchor='bottom',
                                text=titulo,
                                font=dict(family='Arial',
                                          size=20,
                                          color='rgb(37,37,37)'),
                                showarrow=False))

  annotations.append(dict(xref='paper', yref='paper', x=0, y=1.1,
                                xanchor='left', yanchor='top',
                                text=subtitulo,
                                font=dict(family='Arial',
                                          size=12,
                                          color='rgb(150,150,150)'),
                                showarrow=False))

  fig.update_layout(template='plotly_white',
                    annotations=annotations,
                    autosize=False,
                    width=800,height=500,
                    legend=dict(x=0.01,y=1,traceorder="reversed",
                                font=dict(family="Arial",size=12,color="Dimgray"),
                                bgcolor="white",bordercolor="Dimgray",borderwidth=2)
                    )  

  fig.update_yaxes(nticks=8,
                  automargin=True,
                  showgrid=False,
                  showline=True,
                  zeroline=False,
                  linecolor='Dimgray',
                  title=unidade_y,
                  title_font=dict(size=16, family='Arial', color='Dimgray'))

  fig.update_xaxes(nticks=5,
                  showgrid=False,
                  showline=True,
                  linecolor='Dimgray',
                  title=unidade_x,
                    title_font=dict(size=16, family='Arial', color='Dimgray'))

  

  fig.write_image("/content/sample_data/Etapa Final/evolucao.png")

  df_base=series[cliente]['Acum. Total']
  titulo='Rentabilidade acumulada desde o início'
  subtitulo='Solicitante: {}'.format(cliente)
  unidade_y='Rentabilidade (%)'
  unidade_x='Data'

  fig = go.Figure()
  n=0
  annotations = []
  fig.add_trace(go.Bar(x=df_base.columns,
                          y=df_base.iloc[-1]*100,
                          marker_color='Dimgray',
                          showlegend=False,
                          ))


  annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.10,
                                xanchor='left', yanchor='bottom',
                                text=titulo,
                                font=dict(family='Arial',
                                          size=20,
                                          color='rgb(37,37,37)'),
                                showarrow=False))

  annotations.append(dict(xref='paper', yref='paper', x=0, y=1.1,
                                xanchor='left', yanchor='top',
                                text=subtitulo,
                                font=dict(family='Arial',
                                          size=12,
                                          color='rgb(150,150,150)'),
                                showarrow=False))

  fig.update_layout(template='plotly_white',
                    annotations=annotations,xaxis_tickangle=-45,
                    autosize=False,
                    width=800,height=500,
                    legend=dict(x=0.01,y=1,traceorder="reversed",
                                font=dict(family="Arial",size=12,color="Dimgray"),
                                bgcolor="white",bordercolor="Dimgray",borderwidth=2)
                    )  
  datas_iniciais = []
  for cnpj in df_base:
    datas_iniciais.append(df_base[cnpj].dropna().index[0].strftime("%d-%m-%Y"))

  fig.update_traces(texttemplate=datas_iniciais, textposition='outside')


  fig.update_yaxes(nticks=8,
                  automargin=True,
                  showgrid=False,
                  showline=True,
                  zeroline=False,
                  linecolor='Dimgray',
                  title=unidade_y,
                  title_font=dict(size=16, family='Arial', color='Dimgray'))

  fig.update_xaxes(nticks=len(df_base.columns),
                  showgrid=False,
                  showline=True,
                  linecolor='Dimgray',
                  title=unidade_x,
                  title_font=dict(size=16, family='Arial', color='Dimgray'))

  

  fig.write_image("/content/sample_data/Etapa Final/ret_total.png")

  document = Document('/content/drive/MyDrive/Colab Notebooks/Carteira Quant/Gera Carteiras Dia/modelo_relatorios_fis.docx')

  run = document.add_paragraph().add_run()
  font = run.font
  font.color.rgb = RGBColor(0x0, 0x0, 0x0)
  font.size = Pt(12)

  document.add_heading('Sobre esse informativo:', level=1)
  paragraph =  document.add_paragraph('Esse informativo foi gerado a partir do formulário diponível no site www.comoserumcientistadedados.com.br, sob o menu Carteira Quant. O principal objetivo desse informativo é facilitar a descoberta de fundos de investimentos para os investidores leigos, não sendo uma recomendação. Cada relatório é único e busca os investimentos que mais correspondam às expectativas dos solicitantes.')
  paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY


  document.add_heading('Informações do solicitante:', level=1)
  paragraph =  document.add_paragraph('Email: '+clientes_dia.loc[cliente].name)
  paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

  
  n=0
  for info in clientes_dia.loc[cliente].index:
    paragraph =  document.add_paragraph(str(clientes_dia.loc[cliente].index[n])+' '+str(clientes_dia.loc[cliente].values[n]))
    paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    n+=1

  document.add_heading('Sobre a possibilidade de alcance do objetivo:', level=1)
  paragraph =  document.add_paragraph(controle_parcela[cliente])
  paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

  
  #acima_cdi = (df_base.iloc[-1] > ((1+(cdi_graf['valor'].loc[df_base[item].dropna().index[0]:]/100)).cumprod()-1).iloc[-1]).loc[(df_base.iloc[-1] > ((1+(cdi_graf['valor'].loc[df_base[item].index[0]:]/100)).cumprod()-1).iloc[-1]) == True].index
  

  document.add_heading('Fundos de Investimento:', level=1)
  
  if len(acima_cdi) != 0:
    for cnpj in cnpjs_solicitantes[cliente][acima_cdi]:
      paragraph = document.add_paragraph(cnpj+' - '+cadastrais.loc[cnpj]['DENOM_SOCIAL'])
      paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY_MED  
    document.add_heading('Gráficos: ', level=1)
    paragraph = document.add_picture('/content/sample_data/Etapa Final/ret_total.png',width=Inches(7))
    paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    paragraph = document.add_picture('/content/sample_data/Etapa Final/evolucao.png',width=Inches(7))
    paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    document.save(salvar_em+'/'+cliente+'.docx')
  else:
    paragraph = document.add_paragraph('Os fundos filtrados não superaram o CDI. Investimentos pré-fixados e alguns pós-fixados satisfarão suas necessidades. Consulte um especialista.')
    paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY_MED  
    document.save(salvar_em+'/'+cliente+'.docx')


os arquivos serão salvos em: /content/sample_data/Clientes do Dia
Foram encontrados 2197 fundos da sua instituição
os arquivos serão salvos em: /content/sample_data/Clientes do Dia
os arquivos serão salvos em: /content/sample_data/Clientes do Dia
os arquivos serão salvos em: /content/sample_data/Etapa Final
os arquivos serão salvos em: /content/sample_data/Etapa Final/Relatorio
